# Monk1 
Grid Search for Monk1 dataset model selection


In [1]:
from exclusiveAI.ConfiguratorGen import ConfiguratorGen
from exclusiveAI.datasets.monk import read_monk1
from exclusiveAI.components.Validation import *
from exclusiveAI.utils import one_hot_encoding
from exclusiveAI.components.CallBacks import *
from exclusiveAI.Composer import Composer
import pandas as pd
import numpy as np

#### Read Monk1 dataset 

In [2]:
training_data, training_labels, test_data, test_labels = read_monk1()

One-Hot Encoding Training Dataset

In [3]:
training_data = one_hot_encoding(training_data)

One-Hot Encoding Test Dataset

In [4]:
test_data = one_hot_encoding(test_data)

Early Stopping Callback with 50 epochs patience limit

In [5]:
patience_limit = 10
early_stopping = EarlyStoppingCallback(patience_limit=patience_limit, eps=1e-3)

Configuration Generator Grid Search for better hyperparameters selection

In [ ]:
regularizations = np.arange(0, 0.001, 0.0001)
learning_rates = np.arange(0.01, 0.5, 0.01)
number_of_units = range(1, 5, 1)
number_of_layers = range(1, 3, 1) 
initializers = ["uniform", "gaussian"]
momentums = np.arange(0, 0.999, 0.001)
activations = ["sigmoid"]

myConfigurator = ConfiguratorGen(random=False, regularizations=regularizations, learning_rates=learning_rates,
                                 loss_function=['mse'], optimizer=['sgd'],
                                 activation_functions=activations, nesterov=True,
                                 number_of_units=number_of_units, number_of_layers=number_of_layers,
                                 momentums=momentums, initializers=initializers, input_shapes=training_data.shape, verbose=False,
                                 callbacks=[early_stopping], output_activation='sigmoid'
                                 )

2nd for:  96%|█████████▌| 3759477/3916080 [04:40<00:03, 45665.20it/s]

HoldOut validation with our configurations

In [ ]:
batch_size=32
epochs=200

myValidation = HoldOut(models=myConfigurator, input=training_data, target=training_labels, debug=True)
configs = myValidation.hold_out(all_models=True, batch_size=batch_size, epochs=epochs)

In [ ]:
configs = pd.DataFrame(configs)
# Save as json
configs.to_json('monk1_configurations.json')